# FFNN neural network for traffic flow predictions

This notebook contains the code for applying neural network models to smart city data <br>


In [ ]:
# GPU check:

import tensorflow as tf
tf.test.gpu_device_name()


'/device:GPU:0'

In [ ]:
from tensorflow.python.client import device_lib
print("Show System RAM Memory: \n \n")
!cat /proc/meminfo | egrep "MemTotal"
device_lib.list_local_devices()

Show System RAM Memory: 
 

MemTotal:       26751688 kB


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3884396923609917848, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 18381392720526097635
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1786750086594131231
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15695549568
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9943357225911042361
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

Tutorial about google colab and GPU access: <br>
https://www.youtube.com/watch?v=f1UK8KPt-KU

In [ ]:
# this allows for accessing files stored in your google drive using the path "/gdrive/My Drive/"
# mounting google drive locally:

from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
# importing local data to google colab:
#from google.colab import files
#uploaded = files.upload()

Saving taxi_series_H to taxi_series_H


$\textbf{Background:}$ Tensors are data structures that you can think of as multi-dimensional arrays. Tensors are represented as n-dimensional arrays of base dataypes such as a string or integer -- they provide a way to generalize vectors and matrices to higher dimensions. The shape of a Tensor defines its number of dimensions and the size of each dimension. The rank of a Tensor provides the number of dimensions. Scalars can be used to create 0-d Tensors. Vectors and lists can be used to create 1-d Tensors. Matrices can be used to create 2-d or higher rank Tensors. The shape of a Tensor provides the number of elements in each Tensor dimension.

$\textbf{Neural Networks in Tensorflow:}$ We can also define neural networks in TensorFlow. TensorFlow uses a high-level API called Keras that provides a powerful, intuitive framework for building and training deep learning models. <br> 
Tensors can flow through abstract types called $\textit{Layers}$ -- the building blocks of neural networks. Layers implement common neural networks operations, and are used to update weights, compute losses, and define inter-layer connectivity <br>
<br>
Conveniently, TensorFlow has defined a number of Layers that are commonly used in neural networks, for example a Dense. Now, instead of using a single Layer to define our simple neural network, we'll use the Sequential model from Keras and a single Dense layer to define our network. With the Sequential API, you can readily create neural networks by stacking together layers like building blocks.

# Implementation

In [ ]:
## -- Packages  -- ##

# General
import pandas as pd
import numpy as np

# Time formatting
import datetime

# Load and save data
import pickle
# progress bar
from tqdm import tqdm

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline
#import tikzplotlib as tkz

In [ ]:
##  NN libaries ##
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt


## ML libraries ##
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import MinMaxScaler

### Load data

In [ ]:
#load taxi data. Generated in notebook 'taxi_trips'
filename = '/gdrive/My Drive/Colab Notebooks/taxi_series_H'
infile = open(filename,'rb')
taxi_ts = pickle.load(infile)
infile.close()

In [ ]:
taxi_ts.shape

(8760,)

### Preprocessing

In [ ]:
## Remove last 24 hours as well as the part of test set not being used for main results
taxi_ts = taxi_ts.iloc[0:-192]
taxi_ts.shape

(8568,)

In [ ]:
## Set paramaters
input_lags = 60 # 2 and a half times the seasonal period
output_lags = 24 # we predict 24 hours ahead
n_test = 24 # output_lags  (changed)


In [ ]:
## Split data in train and test set
train = taxi_ts[0:-n_test]
test = taxi_ts[-n_test:]
print(train.shape)
print(test.shape)

(8544,)
(24,)


In [ ]:
## Create lagged values for both input and output window (24)
data = train.copy()
n_train = len(data)

##Create lagged values for input
df = pd.DataFrame()
for i in range(input_lags,0,-1):
    df['t-' + str(i)] = data.shift(i)

##Create lagged values for output
for j in range(0,output_lags,1):
    df['t+' + str(j)] = data.shift(-j)
    
df = df[input_lags:(n_train-output_lags+1)]

In [ ]:
df.head()

,t-60,t-59,t-58,t-57,t-56,t-55,t-54,t-53,t-52,t-51,t-50,t-49,t-48,t-47,t-46,t-45,t-44,t-43,t-42,t-41,t-40,t-39,t-38,t-37,t-36,t-35,t-34,t-33,t-32,t-31,t-30,t-29,t-28,t-27,t-26,t-25,t-24,t-23,t-22,t-21,...,t-16,t-15,t-14,t-13,t-12,t-11,t-10,t-9,t-8,t-7,t-6,t-5,t-4,t-3,t-2,t-1,t+0,t+1,t+2,t+3,t+4,t+5,t+6,t+7,t+8,t+9,t+10,t+11,t+12,t+13,t+14,t+15,t+16,t+17,t+18,t+19,t+20,t+21,t+22,t+23
tpep_pickup_datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-07-03 12:00:00,11510.0,9596.0,7984.0,5919.0,4414.0,2251.0,2725.0,3720.0,5547.0,8934.0,10842.0,12045.0,13286.0,14658.0,14261.0,13059.0,12673.0,14327.0,13896.0,12740.0,11350.0,11197.0,9885.0,8741.0,4915.0,2747.0,1652.0,1173.0,1318.0,2682.0,6690.0,9618.0,12715.0,13279.0,12110.0,12133.0,13245.0,14139.0,14624.0,14278.0,...,12759.0,12897.0,11317.0,8075.0,4866.0,2737.0,1587.0,1156.0,1172.0,2520.0,6333.0,9554.0,12878.0,13005.0,12508.0,12509.0,14326,16733.0,15450.0,14738.0,14632.0,16585.0,16488.0,12862.0,11051.0,11540.0,11029.0,10140.0,8339.0,6441.0,4764.0,3385.0,2489.0,1651.0,2388.0,3018.0,3847.0,5269.0,6621.0,8176.0
2018-07-03 13:00:00,9596.0,7984.0,5919.0,4414.0,2251.0,2725.0,3720.0,5547.0,8934.0,10842.0,12045.0,13286.0,14658.0,14261.0,13059.0,12673.0,14327.0,13896.0,12740.0,11350.0,11197.0,9885.0,8741.0,4915.0,2747.0,1652.0,1173.0,1318.0,2682.0,6690.0,9618.0,12715.0,13279.0,12110.0,12133.0,13245.0,14139.0,14624.0,14278.0,13545.0,...,12897.0,11317.0,8075.0,4866.0,2737.0,1587.0,1156.0,1172.0,2520.0,6333.0,9554.0,12878.0,13005.0,12508.0,12509.0,14326.0,16733,15450.0,14738.0,14632.0,16585.0,16488.0,12862.0,11051.0,11540.0,11029.0,10140.0,8339.0,6441.0,4764.0,3385.0,2489.0,1651.0,2388.0,3018.0,3847.0,5269.0,6621.0,8176.0,9131.0
2018-07-03 14:00:00,7984.0,5919.0,4414.0,2251.0,2725.0,3720.0,5547.0,8934.0,10842.0,12045.0,13286.0,14658.0,14261.0,13059.0,12673.0,14327.0,13896.0,12740.0,11350.0,11197.0,9885.0,8741.0,4915.0,2747.0,1652.0,1173.0,1318.0,2682.0,6690.0,9618.0,12715.0,13279.0,12110.0,12133.0,13245.0,14139.0,14624.0,14278.0,13545.0,14945.0,...,11317.0,8075.0,4866.0,2737.0,1587.0,1156.0,1172.0,2520.0,6333.0,9554.0,12878.0,13005.0,12508.0,12509.0,14326.0,16733.0,15450,14738.0,14632.0,16585.0,16488.0,12862.0,11051.0,11540.0,11029.0,10140.0,8339.0,6441.0,4764.0,3385.0,2489.0,1651.0,2388.0,3018.0,3847.0,5269.0,6621.0,8176.0,9131.0,10124.0
2018-07-03 15:00:00,5919.0,4414.0,2251.0,2725.0,3720.0,5547.0,8934.0,10842.0,12045.0,13286.0,14658.0,14261.0,13059.0,12673.0,14327.0,13896.0,12740.0,11350.0,11197.0,9885.0,8741.0,4915.0,2747.0,1652.0,1173.0,1318.0,2682.0,6690.0,9618.0,12715.0,13279.0,12110.0,12133.0,13245.0,14139.0,14624.0,14278.0,13545.0,14945.0,16542.0,...,8075.0,4866.0,2737.0,1587.0,1156.0,1172.0,2520.0,6333.0,9554.0,12878.0,13005.0,12508.0,12509.0,14326.0,16733.0,15450.0,14738,14632.0,16585.0,16488.0,12862.0,11051.0,11540.0,11029.0,10140.0,8339.0,6441.0,4764.0,3385.0,2489.0,1651.0,2388.0,3018.0,3847.0,5269.0,6621.0,8176.0,9131.0,10124.0,9832.0
2018-07-03 16:00:00,4414.0,2251.0,2725.0,3720.0,5547.0,8934.0,10842.0,12045.0,13286.0,14658.0,14261.0,13059.0,12673.0,14327.0,13896.0,12740.0,11350.0,11197.0,9885.0,8741.0,4915.0,2747.0,1652.0,1173.0,1318.0,2682.0,6690.0,9618.0,12715.0,13279.0,12110.0,12133.0,13245.0,14139.0,14624.0,14278.0,13545.0,14945.0,16542.0,14673.0,...,4866.0,2737.0,1587.0,1156.0,1172.0,2520.0,6333.0,9554.0,12878.0,13005.0,12508.0,12509.0,14326.0,16733.0,15450.0,14738.0,14632,16585.0,16488.0,12862.0,11051.0,11540.0,11029.0,10140.0,8339.0,6441.0,4764.0,3385.0,2489.0,1651.0,2388.0,3018.0,3847.0,5269.0,6621.0,8176.0,9131.0,10124.0,9832.0,9478.0


In [ ]:
## splitting the training set into labels and features
X_train = df.iloc[:,:input_lags] # from the beginning to input_lags
Y_train = df.iloc[:,input_lags:] # from input_lags to the end

## Use the last window of the training set as the features for the test set. This requires a combination of 
## X_train and Y_train.
X_test = X_train.iloc[len(X_train) - 1,:][output_lags:]
X_test = X_test.append(Y_train.iloc[len(Y_train) - 1,:]).values.reshape(1,input_lags)
Y_test = test[:output_lags].values.reshape(1,output_lags)

X_train = X_train.values # 54 steps back (54 lags)
Y_train = Y_train.values # 24 steps ahead

print("X_train: " + "type: " + str(type(X_train)) + "\tshape: " + str(X_train.shape))
print("Y_train: " + "type: " + str(type(Y_train)) + "\tshape: " + str(Y_train.shape))
print("X_test: " + "type: " + str(type(X_test)) + "\tshape: " + str(X_test.shape))
print("Y_test: " + "type: " + str(type(Y_test)) + "\tshape: " + str(Y_test.shape))

X_train: type: <class 'numpy.ndarray'>	shape: (8461, 60)
Y_train: type: <class 'numpy.ndarray'>	shape: (8461, 24)
X_test: type: <class 'numpy.ndarray'>	shape: (1, 60)
Y_test: type: <class 'numpy.ndarray'>	shape: (1, 24)


$\textbf{When Should You Use Normalization And Standardization:}$

Normalization is a good technique to use when you do not know the distribution of your data or when you know the distribution is not Gaussian (a bell curve). Normalization is useful when your data has varying scales and the algorithm you are using does not make assumptions about the distribution of your data, such as k-nearest neighbors and artificial neural networks.

In [ ]:
## normalizing the data

# MinMaxScaler() transforms features by scaling each feature to a given range (given by feature_range())
# The cost of having this bounded range is that we will end up with smaller standard deviations, which can 
# suppress the effect of outliers. Thus MinMax Scalar is sensitive to outliers

#scaler = MinMaxScaler(feature_range=(0, 1))
#df_x = df.iloc[:,0:24]
# the method fit_transform() computes the min and the max used for scaling and then carries out the transformation
#df_x_scaled = scaler.fit_transform(df_x)
# later, inverse_transform() can be used to undo the scaling to the feature_range

# normalizing the entire dataset
#df_normalized = scaler.fit_transform(df)

# Building the FFNN neural network model


# Method 1 (Keras)

Tensorflow 2.0 Impelementation <br>
(tf.keras is TensorFlow's implementation of the Keras API specification. This is a high-level API to build and train models that includes first-class support for TensorFlow-specific functionality)


TF 2 keras RNN tutorial
https://www.tensorflow.org/guide/keras/rnn <br>
TF 2 time series forecasting tutorial
https://www.tensorflow.org/tutorials/structured_data/time_series

In [ ]:
# configuring the inputs for the model
# For Keras, the input has to be in the shape (samples, time steps, features)
# 24 timestep with n features where n is equal to the shape of column [1] of X_train or X_test

X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [ ]:
X_train.shape

(589, 1, 60)

In [ ]:
X_test.shape

(1, 1, 60)

In [ ]:
# creating a leaky_relu activation function

def my_leaky_relu(x):
    return tf.nn.leaky_relu(x)

## Fitting the FFNN model with fixed hyperparameters and 3 Dense (fully connected layers)


In [ ]:


# fitting the FFNN model with fixed hyperparameters
FFNN_model = tf.keras.Sequential()


FFNN_model.add(tf.keras.layers.Dense(units=128, input_shape = (1,60)))
FFNN_model.add(tf.keras.layers.Dense(units=64))


# output layer 
FFNN_model.add(tf.keras.layers.Dense(24))
# the compile() method configures the model for training
FFNN_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.01),
              loss=tf.keras.losses.mean_squared_error,
              metrics=['mean_squared_error'])
FFNN_model.fit(X_train, Y_train, epochs=100, batch_size=100, verbose=1)
    


Epoch 1/100
85/85 [==============================] - 0s 2ms/step - loss: 25281768.0000 - mean_squared_error: 25281768.0000
Epoch 2/100
85/85 [==============================] - 0s 2ms/step - loss: 6624039.5000 - mean_squared_error: 6624039.5000
Epoch 3/100
85/85 [==============================] - 0s 2ms/step - loss: 6450304.5000 - mean_squared_error: 6450304.5000
Epoch 4/100
85/85 [==============================] - 0s 2ms/step - loss: 6228031.0000 - mean_squared_error: 6228031.0000
Epoch 5/100
85/85 [==============================] - 0s 2ms/step - loss: 6102186.5000 - mean_squared_error: 6102186.5000
Epoch 6/100
85/85 [==============================] - 0s 2ms/step - loss: 6101338.5000 - mean_squared_error: 6101338.5000
Epoch 7/100
85/85 [==============================] - 0s 2ms/step - loss: 6427315.5000 - mean_squared_error: 6427315.5000
Epoch 8/100
85/85 [==============================] - 0s 2ms/step - loss: 6011868.0000 - mean_squared_error: 6011868.0000
Epoch 9/100
85/85 [===========

In [ ]:
FFNN_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1, 128)            7808      
_________________________________________________________________
dense_3 (Dense)              (None, 1, 64)             8256      
_________________________________________________________________
dense_4 (Dense)              (None, 1, 24)             1560      
Total params: 17,624
Trainable params: 17,624
Non-trainable params: 0
_________________________________________________________________


## Undoing the normalization, making predictions and computing the test error

In [ ]:
# making predictions (regular FFNN)
y_train_pred = FFNN_model.predict(X_train)
y_test_pred = FFNN_model.predict(X_test)

In [ ]:
# MSE FFNN forecast
mse_FFNN = mean_squared_error(Y_test, y_test_pred)
mse_FFNN

6213276.503940724

In [ ]:
# RMSE FFNN forecast
rmse_FFNN = sqrt(mse_FFNN)
rmse_FFNN

2492.6444800534077

In [ ]:
# mae FFNN
mae_FFNN = mean_absolute_error(Y_test, y_test_pred)
mae_FFNN

1913.3059794108074

In [ ]:
# custom MAPE function
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

y_true = Y_test
y_pred = y_test_pred

In [ ]:
# MAPE FFNN
mean_absolute_percentage_error(y_true,y_pred)

34.52555592649445

## Prediction intervals using quantile regression

1 single PI network

In [ ]:
class GRU_PI_model(tf.keras.Model):
    def __init__(self, input_shape = (1,54), output_shape = 24, alpha = 0.95, batch_size = 200):
        super(GRU_PI_model, self).__init__()
        
        self.in_shape = input_shape
        self.out_shape = output_shape
        self.lower_alpha = (1-alpha)/2
        self.upper_alpha = (1-alpha)/2 + alpha
        self.batch_size = batch_size #move batch size to fit?
        
        self.loss_history = []
        
        ##define architechture 
        self.L1 = tf.keras.layers.GRU(units=1000, activation = my_leaky_relu, input_shape = (self.in_shape,None), dropout = 0.0, return_sequences= False)
        #self.L2 = tf.keras.layers.Dense(units =24, activation = "relu")
        self.output_lowerPI = tf.keras.layers.Dense(self.out_shape)
        self.output_upperPI = tf.keras.layers.Dense(self.out_shape)
        
        ##define optimizer
        self.train_opt = tf.keras.optimizers.Adam(learning_rate=0.001)
        
    def call(self, X): # use predict() for forward pass outside fit
        #X = tf.dtypes.cast(X, tf.float32) 
        
        X = self.L1(X)
        output_lowerPI = self.output_lowerPI(X) 
        output_upperPI = self.output_upperPI(X)
        
        combined_output = [output_lowerPI, output_upperPI]
        return combined_output
    
    def get_loss(self, X, Y):
        
        Y_hat = self.call(X)
        output_lowerPI = Y_hat[0]
        output_upperPI = Y_hat[1]
        
        error_lower = tf.subtract(Y, output_lowerPI)
        loss_lower = tf.math.reduce_mean(tf.math.maximum(self.lower_alpha * error_lower, (self.lower_alpha - 1) * error_lower))
        
        error_upper = tf.subtract(Y, output_upperPI)
        loss_upper = tf.math.reduce_mean(tf.math.maximum(self.upper_alpha * error_upper, (self.upper_alpha - 1) * error_upper))
        
        combined_loss = tf.add(loss_lower,loss_upper)
        return combined_loss
        
    def get_grad(self, X, Y):
        with tf.GradientTape() as tape:
            tape.watch(self.trainable_variables)
            
            L = self.get_loss(X, Y)
        g = tape.gradient(L, self.trainable_variables)
        
        return L, g    
    
    def train_step(self, X, Y):
        L, g = self.get_grad(X, Y)
    
        self.train_opt.apply_gradients(zip(g, self.trainable_variables))
        
        return L
        
    def fit(self, X, Y, epochs = 300): 
        X = tf.dtypes.cast(X, tf.float32) ##layers use float 32, so cast data to fit
        Y = tf.dtypes.cast(Y, tf.float32) ##layers use float 32, so cast data to fit
        
        if hasattr(tqdm, '_instances'): tqdm._instances.clear() ## clear load bar if it is not empty

        loadbar = tqdm(range(epochs)) ##define loadbar with loop range, which is number of epochs
        for epoch in loadbar:
            loadbar.set_description("Epoch {}".format(epoch + 1))
            
            batch_losses = []
            for idx in range(0, X.shape[0], self.batch_size): ## get batches of size batch_size. NOT shuffled
                batch_X = X[idx : min(idx + self.batch_size, X.shape[0]),:]
                batch_Y = Y[idx : min(idx + self.batch_size, Y.shape[0]),:]
                L = self.train_step(batch_X, batch_Y)
                batch_losses.append(L.numpy())
        
            epoch_loss = np.mean(batch_losses)
            self.loss_history.append(np.mean(batch_losses))

In [ ]:
model_95 = GRU_PI_model(input_shape =  (1,54), output_shape = 24, batch_size = 200, alpha = 0.95)

In [ ]:
model_95.fit(X_train, Y_train, epochs = 300)

Epoch 300: 100%|██████████| 300/300 [00:16<00:00, 18.46it/s]


In [ ]:
## make predictions and format them for plotting
predictions_95 = model_95.predict(X_test)
lower_PI_preds_95 = predictions_95[0]
upper_PI_preds_95 = predictions_95[1]


lower_PI_preds_95 = pd.Series(lower_PI_preds_95.reshape(output_lags,))
lower_PI_preds_95.index = test.iloc[:output_lags].index
upper_PI_preds_95 = pd.Series(upper_PI_preds_95.reshape(output_lags,))
upper_PI_preds_95.index = test.iloc[:output_lags].index


2 separate PI networks

In [ ]:
# quantile regression loss = tilted loss = pinball loss
def tilted_loss(q,y,f):
    e = (y-f)
    return K.mean(K.maximum(q*e, (q-1)*e), axis=-1)

In [ ]:
## build lower bound with different loss (95% PI)
q = 0.025



# fitting the FFNN model with fixed hyperparameters
lower_FFNN = tf.keras.Sequential()


lower_FFNN.add(tf.keras.layers.Dense(units=500, input_shape = (1,60)))
lower_FFNN.add(tf.keras.layers.Dense(units=250))


# output layer 
lower_FFNN.add(tf.keras.layers.Dense(24))
# the compile() method configures the model for training
lower_FFNN.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
              loss=lambda y,f: tilted_loss(q,y,f))
lower_FFNN.fit(X_train, Y_train, epochs=300, batch_size=300, verbose=1)


Epoch 1/300
29/29 [==============================] - 0s 2ms/step - loss: 365.1037
Epoch 2/300
29/29 [==============================] - 0s 2ms/step - loss: 164.6340
Epoch 3/300
29/29 [==============================] - 0s 2ms/step - loss: 149.4442
Epoch 4/300
29/29 [==============================] - 0s 2ms/step - loss: 140.3334
Epoch 5/300
29/29 [==============================] - 0s 2ms/step - loss: 139.2760
Epoch 6/300
29/29 [==============================] - 0s 2ms/step - loss: 139.0358
Epoch 7/300
29/29 [==============================] - 0s 2ms/step - loss: 136.9417
Epoch 8/300
29/29 [==============================] - 0s 2ms/step - loss: 135.6466
Epoch 9/300
29/29 [==============================] - 0s 2ms/step - loss: 135.6409
Epoch 10/300
29/29 [==============================] - 0s 2ms/step - loss: 131.9883
Epoch 11/300
29/29 [==============================] - 0s 2ms/step - loss: 131.3001
Epoch 12/300
29/29 [==============================] - 0s 2ms/step - loss: 130.7982
Epoch 13/300


In [ ]:
## build upper bound with different loss (95% PI)
q = 0.975


# fitting the FFNN model with fixed hyperparameters
upper_FFNN = tf.keras.Sequential()


upper_FFNN.add(tf.keras.layers.Dense(units=500, input_shape = (1,60)))
upper_FFNN.add(tf.keras.layers.Dense(units=250))


# output layer 
upper_FFNN.add(tf.keras.layers.Dense(24))
# the compile() method configures the model for training
upper_FFNN.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
              loss=lambda y,f: tilted_loss(q,y,f))
upper_FFNN.fit(X_train, Y_train, epochs=300, batch_size=300, verbose=1)

Epoch 1/300
29/29 [==============================] - 0s 2ms/step - loss: 1216.7528
Epoch 2/300
29/29 [==============================] - 0s 2ms/step - loss: 246.3846
Epoch 3/300
29/29 [==============================] - 0s 2ms/step - loss: 167.5446
Epoch 4/300
29/29 [==============================] - 0s 2ms/step - loss: 158.6900
Epoch 5/300
29/29 [==============================] - 0s 2ms/step - loss: 152.4806
Epoch 6/300
29/29 [==============================] - 0s 2ms/step - loss: 149.4104
Epoch 7/300
29/29 [==============================] - 0s 2ms/step - loss: 152.3887
Epoch 8/300
29/29 [==============================] - 0s 2ms/step - loss: 148.3315
Epoch 9/300
29/29 [==============================] - 0s 2ms/step - loss: 149.7787
Epoch 10/300
29/29 [==============================] - 0s 2ms/step - loss: 150.5515
Epoch 11/300
29/29 [==============================] - 0s 2ms/step - loss: 150.1481
Epoch 12/300
29/29 [==============================] - 0s 2ms/step - loss: 148.7434
Epoch 13/300

In [ ]:
# predictions from PI models

lower_FFNN = lower_FFNN.predict(X_test)
upper_FFNN = upper_FFNN.predict(X_test)

PI measure calculations

In [ ]:
def PCIP(upper, lower, test_set):
    n = len(Y_test[0,:])
    count = 0
    PCIP = 0
    
    for i in range(n):
        if (upper[0,i] > test_set[0,i] and lower[0,i] < test_set[0,i]):
            count = count + 1 
            
    PCIP = count/n
    return PCIP

In [ ]:
# FFFNN PI measures 
PCIP(upper = upper_FFNN, lower = lower_FFNN, test_set = Y_test)

0.9583333333333334

In [ ]:
def MPIW(upper, lower):
    n = len(upper[0,:])
    diff = 0
    MPIW = 0
    
    for i in range(n):
        diff = diff + (upper[0,i] - lower[0,i])
    
    MPIW = diff/n
    return MPIW

In [ ]:
# MPIW for the FFNN PIs
MPIW(upper = upper_FFNN, lower = lower_FFNN)

7449.2864990234375